# minGPT

**Note:** The `autoreload` extension allows the interpreter to reload modules every time a cell is executed. This is useful when editing the code in a module. The following cell enables the extension and downloads the minGPT package from Github. You can now double-click on a file like model.py, edit its contents, and press Ctrl+S to save it. If you then re-run the notebook cells, including those that create an object of the corresponding class, you will see the changes reflected. Note that the next cell should *only be executed once*, as running `pip install` again will overwrite the modified contents of the module.

Recall that changes in the files (except the notebook itself) are not persistent unless you connect them to your Google Drive account.

In [2]:
%load_ext autoreload
%autoreload 2
%pip install -e 'git+https://github.com/karpathy/minGPT.git@37baab71b9abea1b76ab957409a1cc2fbfba8a26#egg=mingpt'

# Fix this issue: https://github.com/karpathy/minGPT/issues/120
!sed -i '200s/.*/        assert len(keys) == len([k for k in sd if not k.endswith(".attn.bias")])/' /content/src/mingpt/mingpt/model.py


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Obtaining mingpt from git+https://github.com/karpathy/minGPT.git@37baab71b9abea1b76ab957409a1cc2fbfba8a26#egg=mingpt
  Skipping because already up-to-date.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached torch-2.9.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached filelock-3.20.2-py3-none-any.whl.metadata (2.1 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.6.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.12.0-py3-none-any.whl.metadata (10 kB)
 

Add module's location to PYTHONPATH, which tells your Python interpreter where to search modules for. The previous `pip install -e` changes the variable in a subshell and the interpreter is therefore not aware of the updated value.

In [3]:
import sys
sys.path.append('/content/src/mingpt')

In [4]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 303.4 kB/s eta 0:00:00a 0:00:01
  Using cached pyyaml-6.0.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (2.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.9 MB/s eta 0:00:00
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached charset_normalizer-3.4.4-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (37 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 11.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 8.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 11.2 MB/s eta 0:00:0000:0100:01
Using cached pyyaml-6.0.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl (8

In [6]:
import sys
print(sys.executable)


/home/bledyx/UA/master-ia/TPLN/code/lvl1/lvl2/tpln-practice2/.venv/bin/python


In [3]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from mingpt.model import GPT
from mingpt.utils import set_seed
from mingpt.bpe import BPETokenizer
set_seed(3407)

/home/bledyx/UA/master-ia/TPLN/code/lvl1/lvl2/tpln-practice2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
use_mingpt = True # use minGPT or huggingface/transformers model?
model_type = 'gpt2'
device = 'cuda'

In [4]:
if use_mingpt:
    model = GPT.from_pretrained(model_type)
else:
    model = GPT2LMHeadModel.from_pretrained(model_type)
    model.config.pad_token_id = model.config.eos_token_id # suppress a warning

# ship model to device and set to eval mode
model.to(device)
model.eval();

number of parameters: 124.44M


In [5]:

def generate(prompt='', num_samples=10, steps=20, do_sample=True):

    # tokenize the input prompt into integer input sequence
    if use_mingpt:
        tokenizer = BPETokenizer()
        if prompt == '':
            # to create unconditional samples...
            # manually create a tensor with only the special <|endoftext|> token
            # similar to what openai's code does here https://github.com/openai/gpt-2/blob/master/src/generate_unconditional_samples.py
            x = torch.tensor([[tokenizer.encoder.encoder['<|endoftext|>']]], dtype=torch.long)
        else:
            x = tokenizer(prompt).to(device)
    else:
        tokenizer = GPT2Tokenizer.from_pretrained(model_type)
        if prompt == '':
            # to create unconditional samples...
            # huggingface/transformers tokenizer special cases these strings
            prompt = '<|endoftext|>'
        encoded_input = tokenizer(prompt, return_tensors='pt').to(device)
        x = encoded_input['input_ids']

    # we'll process all desired num_samples in a batch, so expand out the batch dim
    x = x.expand(num_samples, -1)

    # forward the model `steps` times to get samples, in a batch
    y = model.generate(x, max_new_tokens=steps, do_sample=do_sample, top_k=40)

    for i in range(num_samples):
        out = tokenizer.decode(y[i].cpu().squeeze())
        print('-'*80)
        print(out)


In [6]:
generate(prompt='Andrej Karpathy, the Earth representative on', num_samples=10, steps=20)

downloading https://openaipublic.blob.core.windows.net/gpt-2/models/124M/encoder.json to /home/bledyx/.cache/mingpt/encoder.json
downloading https://openaipublic.blob.core.windows.net/gpt-2/models/124M/vocab.bpe to /home/bledyx/.cache/mingpt/vocab.bpe
--------------------------------------------------------------------------------
Andrej Karpathy, the Earth representative on NASA's Juno mission, will also receive this year's award.

While his experience shows that
--------------------------------------------------------------------------------
Andrej Karpathy, the Earth representative on Russia's delegation to the G20 summit, spoke about his work to end climate change in his blog
--------------------------------------------------------------------------------
Andrej Karpathy, the Earth representative on the United Nations Security Council who was asked on Monday to take a position on climate change in order to
----------------------------------------------------------------------------